# Fighting fire with firepower
## Notebook 5: Webscraping
<br>
<br>
<strong>General Assembly Data Science Intensive 7:</strong><br>

> Wayne Chan <br>
> Mariam Javed <br>
> Shawn Syms 

<a name="contents"></a>
## Contents

* <a href="#context">Context</a>
* <a href="#imports">Imports</a>
* <a href="#dataframe-initialization">Dataframe initialization</a>
* <a href="#api-and-webscraping">API and webscraping</a>

<a name="context"></a>
## Context

As part of our efforts to increase our model's performance, we integrated a set of weather measurements. To do this, we used the DarkSky API to capture weather data for the days/times/locations of the fires in a subset of our dataset. This was a time-consuming process. These are the measurements that we integrated into our model's feature set:

> - precipitation intensity<br>
> - precipitation probability<br>
> - temperature high<br>
> - temperature low<br>
> - humidity<br>
> - wind speed<br>
> - wind gust<br>
> - UV index<br>
> - visibility<br>

Ultimately, we found that the weather data was not strongly correlated to our training target (fire size), but we are including the notebook for the sake of completeness of documentation. 

As well, our data originated in a SQL database; we have captured those details here as well.

<div style="text-align: right">(<a href="#contents">home</a>) </div>

<a name="imports"></a>
## Imports

In [2]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import requests
import datetime
import time
pd.set_option('display.max_colwidth', -1)

from bs4 import BeautifulSoup

<div style="text-align: right">(<a href="#contents">home</a>) </div>

<a name="dataframe-initialization"></a>
## Dataframe initialization

In [3]:
#establish a connection with the database file
cnx = sqlite3.connect('kaggle_dataset/FPA_FOD_20170508.sqlite')

Now create our dataframe by running a select query on the DB:

In [3]:
df = pd.read_sql_query("SELECT FIRE_YEAR, DISCOVERY_DATE, DISCOVERY_DOY, DISCOVERY_TIME, STAT_CAUSE_DESCR, LATITUDE, LONGITUDE, STATE, FIRE_CODE, FIRE_SIZE FROM 'Fires'", cnx)
df.head()

,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE,STATE,FIRE_CODE,FIRE_SIZE
0,2005,2453403.5,33,1300,Miscellaneous,40.036944,-121.005833,CA,BJ8K,0.10
1,2004,2453137.5,133,0845,Lightning,38.933056,-120.404444,CA,AAC0,0.25
2,2004,2453156.5,152,1921,Debris Burning,38.984167,-120.735556,CA,A32W,0.10
3,2004,2453184.5,180,1600,Lightning,38.559167,-119.913333,CA,None,0.10
4,2004,2453184.5,180,1600,Lightning,38.559167,-119.933056,CA,None,0.10


In [4]:
df.to_csv('fire.csv')

In [5]:
df['FIRE_YEAR'].unique()

array([2005, 2004, 2006, 2008, 2002, 2007, 2009, 2001, 2003, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2010, 2011, 2012, 2013,
       2014, 2015])

In [11]:
fire_date = datetime.date(2004,5,12)

unixtime = time.mktime(fire_date.timetuple())
unixtime

1084334400.0

In [12]:
d = datetime.date(int(d[6:]),int(d[3:5]),int(d[0:2]))

unixtime = time.mktime(d.timetuple())
unixtime

1084334400.0

In [14]:
df.drop(['DISCOVERY_TIME'], axis = 1, inplace = True)

In [16]:
df['datetime'] = df['FIRE_YEAR'].astype(str) + df['DISCOVERY_DOY'].astype(str)
df.head()

,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE,STATE,FIRE_CODE,FIRE_SIZE,datetime
0,2005,2453403.5,33,Miscellaneous,40.036944,-121.005833,CA,BJ8K,0.10,200533
1,2004,2453137.5,133,Lightning,38.933056,-120.404444,CA,AAC0,0.25,2004133
2,2004,2453156.5,152,Debris Burning,38.984167,-120.735556,CA,A32W,0.10,2004152
3,2004,2453184.5,180,Lightning,38.559167,-119.913333,CA,None,0.10,2004180
4,2004,2453184.5,180,Lightning,38.559167,-119.933056,CA,None,0.10,2004180


In [19]:
df = df.loc[df['FIRE_YEAR'] >= 2010]

In [20]:
df.head()

,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE,STATE,FIRE_CODE,FIRE_SIZE,datetime
1067487,2010,2455335.5,139,Equipment Use,36.766944,-121.303056,CA,None,70.0,2010139
1067488,2010,2455355.5,159,Miscellaneous,36.776944,-121.311111,CA,None,0.5,2010159
1067489,2010,2455359.5,163,Miscellaneous,36.856111,-121.381111,CA,None,0.1,2010163
1067490,2010,2455361.5,165,Miscellaneous,36.818056,-121.391111,CA,None,0.1,2010165
1067491,2010,2455388.5,192,Miscellaneous,36.883056,-121.561944,CA,None,1.0,2010192


In [21]:
new_list = []

for i in df['datetime']:
    
    dt = datetime.datetime.strptime(i, '%Y%j').strftime('%d/%m/%Y')
    new_list.append(dt)
    
df['dmy'] = new_list

In [23]:
df.reset_index(drop = True, inplace = True)
df.head()

,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE,STATE,FIRE_CODE,FIRE_SIZE,datetime,dmy
0,2010,2455335.5,139,Equipment Use,36.766944,-121.303056,CA,None,70.0,2010139,19/05/2010
1,2010,2455355.5,159,Miscellaneous,36.776944,-121.311111,CA,None,0.5,2010159,08/06/2010
2,2010,2455359.5,163,Miscellaneous,36.856111,-121.381111,CA,None,0.1,2010163,12/06/2010
3,2010,2455361.5,165,Miscellaneous,36.818056,-121.391111,CA,None,0.1,2010165,14/06/2010
4,2010,2455388.5,192,Miscellaneous,36.883056,-121.561944,CA,None,1.0,2010192,11/07/2010


In [25]:
new_list = []

for i in df['dmy']:
    #print(type(i))
    d = datetime.date(int(i[6:]),int(i[3:5]),int(i[0:2]))
    #print(type(d))
    unixtime = time.mktime(d.timetuple())

    new_list.append(unixtime)
    
df['unixtime'] = new_list

In [27]:
df['unixtime'] = df['unixtime'].apply(lambda x: int(x))
df.head()

,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE,STATE,FIRE_CODE,FIRE_SIZE,datetime,dmy,unixtime
0,2010,2455335.5,139,Equipment Use,36.766944,-121.303056,CA,None,70.0,2010139,19/05/2010,1274241600
1,2010,2455355.5,159,Miscellaneous,36.776944,-121.311111,CA,None,0.5,2010159,08/06/2010,1275969600
2,2010,2455359.5,163,Miscellaneous,36.856111,-121.381111,CA,None,0.1,2010163,12/06/2010,1276315200
3,2010,2455361.5,165,Miscellaneous,36.818056,-121.391111,CA,None,0.1,2010165,14/06/2010,1276488000
4,2010,2455388.5,192,Miscellaneous,36.883056,-121.561944,CA,None,1.0,2010192,11/07/2010,1278820800


Note: unixtime is as of 4am due to limitations with dataset.

In [35]:
df.reset_index(drop = True, inplace = True)

In [38]:
df.drop(labels=['FIRE_CODE'], axis = 1, inplace = True)
df.head()

,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE,STATE,FIRE_SIZE,datetime,dmy,unixtime
0,2015,2457314.5,292,Miscellaneous,37.69000,-96.97000,KS,0.01,2015292,19/10/2015,1445227200
1,2015,2457314.5,292,Miscellaneous,39.61380,-82.22510,OH,0.51,2015292,19/10/2015,1445227200
2,2015,2457313.5,291,Miscellaneous,34.68201,-106.77655,NM,0.46,2015291,18/10/2015,1445140800
3,2015,2457314.5,292,Miscellaneous,39.73000,-96.71000,KS,25.00,2015292,19/10/2015,1445227200
4,2015,2457314.5,292,Miscellaneous,37.68000,-97.10000,KS,0.01,2015292,19/10/2015,1445227200


<div style="text-align: right">(<a href="#contents">home</a>) </div>

<a name="api-and-webscraping"></a>
## API and webscraping

Looks like the following variables are what we need for our dataframe:

- precip Intensity
- precip prob
- temperature high
- temp low
- humidity
- wind speed
- wind gust
- uvIndex
- visibility

In [ ]:
precip_intensityls = []
precip_probls = []
temp_highls = []
temp_lowls = []
humidityls = []
wind_speedls = []
wind_gustls = []
uv_indexls = []
visibilityls = []

for i in range(len(df)):
    
    #call api
    lat = df['LATITUDE'][i]
    long = df['LONGITUDE'][i]
    dt = df['unixtime'][i]
    key = 'REDACTED: PUT YOUR OWN API KEY HERE'

    url = 'https://api.darksky.net/forecast/{}/{},{},{}'.format(key, lat, long, dt)

    data = requests.get(url)

    file = data.json()
    
    #add desired data to dataframe
    try:
        precip_intensity = file['daily']['data'][0]['precipIntensity']
    except:
        precip_intensity = np.nan
    
    try:
        precip_prob = file['daily']['data'][0]['precipProbability']
    except:
        precip_prob = np.nan
        
    try:
        temp_high = file['daily']['data'][0]['temperatureHigh']
    except:
        temp_high = np.nan
        
    try:
        temp_low = file['daily']['data'][0]['temperatureLow']
    except:
        temp_low = np.nan
        
    try:
        humidity = file['daily']['data'][0]['humidity']
    except:
        humidity = np.nan
        
    try:
        wind_speed = file['daily']['data'][0]['windSpeed']
    except:
        wind_speed = np.nan
        
    try:
        wind_gust = file['daily']['data'][0]['windGust']
    except:
        wind_gust = np.nan
    
    try:
        uv_index = file['daily']['data'][0]['uvIndex']
    except:
        uv_index = np.nan
        
    try:
        visibility = file['daily']['data'][0]['visibility']
    except:
        visibility = np.nan
    
    precip_intensityls.append(precip_intensity)
    precip_probls.append(precip_prob)
    temp_highls.append(temp_high)
    temp_lowls.append(temp_low)
    humidityls.append(humidity)
    wind_speedls.append(wind_speed)
    wind_gustls.append(wind_gust)
    uv_indexls.append(uv_index)
    visibilityls.append(visibility)
    
    print('Completed {} of {}'.format(i, len(df.index)))
    
    time.sleep(2)

Completed 0 of 22000
Completed 1 of 22000
Completed 2 of 22000
Completed 3 of 22000
Completed 4 of 22000
Completed 5 of 22000
Completed 6 of 22000
Completed 7 of 22000
Completed 8 of 22000
Completed 9 of 22000
Completed 10 of 22000
Completed 11 of 22000
Completed 12 of 22000
Completed 13 of 22000
Completed 14 of 22000
Completed 15 of 22000
Completed 16 of 22000
Completed 17 of 22000
Completed 18 of 22000
Completed 19 of 22000
Completed 20 of 22000
Completed 21 of 22000
Completed 22 of 22000
Completed 23 of 22000
Completed 24 of 22000
Completed 25 of 22000
Completed 26 of 22000
Completed 27 of 22000
Completed 28 of 22000
Completed 29 of 22000
Completed 30 of 22000
Completed 31 of 22000
Completed 32 of 22000
Completed 33 of 22000
Completed 34 of 22000
Completed 35 of 22000
Completed 36 of 22000
Completed 37 of 22000
Completed 38 of 22000
Completed 39 of 22000
Completed 40 of 22000
Completed 41 of 22000
Completed 42 of 22000
Completed 43 of 22000
Completed 44 of 22000
Completed 45 of 2200

Completed 361 of 22000
Completed 362 of 22000
Completed 363 of 22000
Completed 364 of 22000
Completed 365 of 22000
Completed 366 of 22000
Completed 367 of 22000
Completed 368 of 22000
Completed 369 of 22000
Completed 370 of 22000
Completed 371 of 22000
Completed 372 of 22000
Completed 373 of 22000
Completed 374 of 22000
Completed 375 of 22000
Completed 376 of 22000
Completed 377 of 22000
Completed 378 of 22000
Completed 379 of 22000
Completed 380 of 22000
Completed 381 of 22000
Completed 382 of 22000
Completed 383 of 22000
Completed 384 of 22000
Completed 385 of 22000
Completed 386 of 22000
Completed 387 of 22000
Completed 388 of 22000
Completed 389 of 22000
Completed 390 of 22000
Completed 391 of 22000
Completed 392 of 22000
Completed 393 of 22000
Completed 394 of 22000
Completed 395 of 22000
Completed 396 of 22000
Completed 397 of 22000
Completed 398 of 22000
Completed 399 of 22000
Completed 400 of 22000
Completed 401 of 22000
Completed 402 of 22000
Completed 403 of 22000
Completed 4

Completed 718 of 22000
Completed 719 of 22000
Completed 720 of 22000
Completed 721 of 22000
Completed 722 of 22000
Completed 723 of 22000
Completed 724 of 22000
Completed 725 of 22000
Completed 726 of 22000
Completed 727 of 22000
Completed 728 of 22000
Completed 729 of 22000
Completed 730 of 22000
Completed 731 of 22000
Completed 732 of 22000
Completed 733 of 22000
Completed 734 of 22000
Completed 735 of 22000
Completed 736 of 22000
Completed 737 of 22000
Completed 738 of 22000
Completed 739 of 22000
Completed 740 of 22000
Completed 741 of 22000
Completed 742 of 22000
Completed 743 of 22000
Completed 744 of 22000
Completed 745 of 22000
Completed 746 of 22000
Completed 747 of 22000
Completed 748 of 22000
Completed 749 of 22000
Completed 750 of 22000
Completed 751 of 22000
Completed 752 of 22000
Completed 753 of 22000
Completed 754 of 22000
Completed 755 of 22000
Completed 756 of 22000
Completed 757 of 22000
Completed 758 of 22000
Completed 759 of 22000
Completed 760 of 22000
Completed 7

Completed 1072 of 22000
Completed 1073 of 22000
Completed 1074 of 22000
Completed 1075 of 22000
Completed 1076 of 22000
Completed 1077 of 22000
Completed 1078 of 22000
Completed 1079 of 22000
Completed 1080 of 22000
Completed 1081 of 22000
Completed 1082 of 22000
Completed 1083 of 22000
Completed 1084 of 22000
Completed 1085 of 22000
Completed 1086 of 22000
Completed 1087 of 22000
Completed 1088 of 22000
Completed 1089 of 22000
Completed 1090 of 22000
Completed 1091 of 22000
Completed 1092 of 22000
Completed 1093 of 22000
Completed 1094 of 22000
Completed 1095 of 22000
Completed 1096 of 22000
Completed 1097 of 22000
Completed 1098 of 22000
Completed 1099 of 22000
Completed 1100 of 22000
Completed 1101 of 22000
Completed 1102 of 22000
Completed 1103 of 22000
Completed 1104 of 22000
Completed 1105 of 22000
Completed 1106 of 22000
Completed 1107 of 22000
Completed 1108 of 22000
Completed 1109 of 22000
Completed 1110 of 22000
Completed 1111 of 22000
Completed 1112 of 22000
Completed 1113 o

Completed 1414 of 22000
Completed 1415 of 22000
Completed 1416 of 22000
Completed 1417 of 22000
Completed 1418 of 22000
Completed 1419 of 22000
Completed 1420 of 22000
Completed 1421 of 22000
Completed 1422 of 22000
Completed 1423 of 22000
Completed 1424 of 22000
Completed 1425 of 22000
Completed 1426 of 22000
Completed 1427 of 22000
Completed 1428 of 22000
Completed 1429 of 22000
Completed 1430 of 22000
Completed 1431 of 22000
Completed 1432 of 22000
Completed 1433 of 22000
Completed 1434 of 22000
Completed 1435 of 22000
Completed 1436 of 22000
Completed 1437 of 22000
Completed 1438 of 22000
Completed 1439 of 22000
Completed 1440 of 22000
Completed 1441 of 22000
Completed 1442 of 22000
Completed 1443 of 22000
Completed 1444 of 22000
Completed 1445 of 22000
Completed 1446 of 22000
Completed 1447 of 22000
Completed 1448 of 22000
Completed 1449 of 22000
Completed 1450 of 22000
Completed 1451 of 22000
Completed 1452 of 22000
Completed 1453 of 22000
Completed 1454 of 22000
Completed 1455 o

...

Completed 15870 of 22000
Completed 15871 of 22000
Completed 15872 of 22000
Completed 15873 of 22000
Completed 15874 of 22000
Completed 15875 of 22000
Completed 15876 of 22000
Completed 15877 of 22000
Completed 15878 of 22000
Completed 15879 of 22000
Completed 15880 of 22000
Completed 15881 of 22000
Completed 15882 of 22000
Completed 15883 of 22000
Completed 15884 of 22000
Completed 15885 of 22000
Completed 15886 of 22000
Completed 15887 of 22000
Completed 15888 of 22000
Completed 15889 of 22000
Completed 15890 of 22000
Completed 15891 of 22000
Completed 15892 of 22000
Completed 15893 of 22000
Completed 15894 of 22000
Completed 15895 of 22000
Completed 15896 of 22000
Completed 15897 of 22000
Completed 15898 of 22000
Completed 15899 of 22000
Completed 15900 of 22000
Completed 15901 of 22000
Completed 15902 of 22000
Completed 15903 of 22000
Completed 15904 of 22000
Completed 15905 of 22000
Completed 15906 of 22000
Completed 15907 of 22000
Completed 15908 of 22000
Completed 15909 of 22000


Completed 16198 of 22000
Completed 16199 of 22000
Completed 16200 of 22000
Completed 16201 of 22000
Completed 16202 of 22000
Completed 16203 of 22000
Completed 16204 of 22000
Completed 16205 of 22000
Completed 16206 of 22000
Completed 16207 of 22000
Completed 16208 of 22000
Completed 16209 of 22000
Completed 16210 of 22000
Completed 16211 of 22000
Completed 16212 of 22000
Completed 16213 of 22000
Completed 16214 of 22000
Completed 16215 of 22000
Completed 16216 of 22000
Completed 16217 of 22000
Completed 16218 of 22000
Completed 16219 of 22000
Completed 16220 of 22000
Completed 16221 of 22000
Completed 16222 of 22000
Completed 16223 of 22000
Completed 16224 of 22000
Completed 16225 of 22000
Completed 16226 of 22000
Completed 16227 of 22000
Completed 16228 of 22000
Completed 16229 of 22000
Completed 16230 of 22000
Completed 16231 of 22000
Completed 16232 of 22000
Completed 16233 of 22000
Completed 16234 of 22000
Completed 16235 of 22000
Completed 16236 of 22000
Completed 16237 of 22000


Completed 16526 of 22000
Completed 16527 of 22000
Completed 16528 of 22000
Completed 16529 of 22000
Completed 16530 of 22000
Completed 16531 of 22000
Completed 16532 of 22000
Completed 16533 of 22000
Completed 16534 of 22000
Completed 16535 of 22000
Completed 16536 of 22000
Completed 16537 of 22000
Completed 16538 of 22000
Completed 16539 of 22000
Completed 16540 of 22000
Completed 16541 of 22000
Completed 16542 of 22000
Completed 16543 of 22000
Completed 16544 of 22000
Completed 16545 of 22000
Completed 16546 of 22000
Completed 16547 of 22000
Completed 16548 of 22000
Completed 16549 of 22000
Completed 16550 of 22000
Completed 16551 of 22000
Completed 16552 of 22000
Completed 16553 of 22000
Completed 16554 of 22000
Completed 16555 of 22000
Completed 16556 of 22000
Completed 16557 of 22000
Completed 16558 of 22000
Completed 16559 of 22000
Completed 16560 of 22000
Completed 16561 of 22000
Completed 16562 of 22000
Completed 16563 of 22000
Completed 16564 of 22000
Completed 16565 of 22000


Completed 16854 of 22000
Completed 16855 of 22000
Completed 16856 of 22000
Completed 16857 of 22000
Completed 16858 of 22000
Completed 16859 of 22000
Completed 16860 of 22000
Completed 16861 of 22000
Completed 16862 of 22000
Completed 16863 of 22000
Completed 16864 of 22000
Completed 16865 of 22000
Completed 16866 of 22000
Completed 16867 of 22000
Completed 16868 of 22000
Completed 16869 of 22000
Completed 16870 of 22000
Completed 16871 of 22000
Completed 16872 of 22000
Completed 16873 of 22000
Completed 16874 of 22000
Completed 16875 of 22000
Completed 16876 of 22000
Completed 16877 of 22000
Completed 16878 of 22000
Completed 16879 of 22000
Completed 16880 of 22000
Completed 16881 of 22000
Completed 16882 of 22000
Completed 16883 of 22000
Completed 16884 of 22000
Completed 16885 of 22000
Completed 16886 of 22000
Completed 16887 of 22000
Completed 16888 of 22000
Completed 16889 of 22000
Completed 16890 of 22000
Completed 16891 of 22000
Completed 16892 of 22000
Completed 16893 of 22000


Completed 17182 of 22000
Completed 17183 of 22000
Completed 17184 of 22000
Completed 17185 of 22000
Completed 17186 of 22000
Completed 17187 of 22000
Completed 17188 of 22000
Completed 17189 of 22000
Completed 17190 of 22000
Completed 17191 of 22000
Completed 17192 of 22000
Completed 17193 of 22000
Completed 17194 of 22000
Completed 17195 of 22000
Completed 17196 of 22000
Completed 17197 of 22000
Completed 17198 of 22000
Completed 17199 of 22000
Completed 17200 of 22000
Completed 17201 of 22000
Completed 17202 of 22000
Completed 17203 of 22000
Completed 17204 of 22000
Completed 17205 of 22000
Completed 17206 of 22000
Completed 17207 of 22000
Completed 17208 of 22000
Completed 17209 of 22000
Completed 17210 of 22000
Completed 17211 of 22000
Completed 17212 of 22000
Completed 17213 of 22000
Completed 17214 of 22000
Completed 17215 of 22000
Completed 17216 of 22000
Completed 17217 of 22000
Completed 17218 of 22000
Completed 17219 of 22000
Completed 17220 of 22000
Completed 17221 of 22000
